In [1]:
import pickle
from models import String, Gamma, Air, SoundBoard
import numpy as np
import scipy.sparse as sp
import cupy as cp
import cupyx
import cupyx.scipy.sparse.linalg
from tqdm import tqdm

from sparse_utils import inv, set_tol, cpu_to_gpu, gpu_to_cpu

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
constants: dict[str, sp.spmatrix | cp.sparse.spmatrix]
string: String
soundboard: SoundBoard
air: Air
gamma: Gamma
constants, (string, soundboard, air, gamma) = pickle.load(open('constants.pkl', 'rb'))

print(f"{constants['M_sh'].shape = }")
print(f"{constants['M_qh'].shape = }")
print(f"{constants['M_Mh'].shape = }")
print(f"{constants['M_ph'].shape = }")
print(f"{constants['M_ah'].shape = }")
print(f"{constants['M_pah'].shape = }")
print(f"{constants['D_h'].shape = }")
print(f"{constants['J_h'].shape = }")
print(f"{constants['H_h'].shape = }")
print(f"{constants['B_w_h'].shape = }")
print(f"{constants['Gh'].shape = }")


constants['M_sh'].shape = (99, 99)
constants['M_qh'].shape = (100, 100)
constants['M_Mh'].shape = (61200, 61200)
constants['M_ph'].shape = (20400, 20400)
constants['M_ah'].shape = (382500, 382500)
constants['M_pah'].shape = (125000, 125000)
constants['D_h'].shape = (99, 100)
constants['J_h'].shape = (20400, 100)
constants['H_h'].shape = (61200, 20400)
constants['B_w_h'].shape = (44761, 20400)
constants['Gh'].shape = (382500, 125000)


In [ ]:
for key in list(constants.keys()):
    if key.startswith('M') and not key.endswith('inverse'):
        if key + '_inverse' not in constants:
            print(key)
            # calculate the inverse
            print(f"Calculating {key}")
            constants[key + '_inverse'] = inv(cp.sparse.coo_matrix(constants[key]))

In [5]:
gpu_to_cpu(constants)
constants['K_h'] = constants["H_h"].T @ constants["M_Mh_inverse"] @ constants["H_h"]
constants['K_h'] = cp.asarray(constants['K_h'].toarray())
constants['omega_n'], _ = cp.linalg.eigh(constants['K_h'])

In [8]:
constants['delta_t'] = 1/ 50000
constants['n_specturm'] = 50

In [9]:
pickle.dump((constants, (string, soundboard, air, gamma)), open('constants.pkl', 'wb'))